In [8]:
!pip install optuna

In [9]:
!pip install haversine

In [10]:
import pandas as pd
import numpy as np
import random

import copy
import pickle
from datetime import datetime

import xgboost as xgb
import xgboost as xg
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import mean_absolute_error as MAE
from sklearn.metrics import median_absolute_error as MAPE
from sklearn.metrics import r2_score  as R2
from xgboost import plot_importance
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from xgboost import XGBRegressor

from sklearn.metrics import r2_score

# Parameter tunning
import optuna
from sklearn.model_selection import KFold

import haversine as hs

from google.colab import drive

import seaborn as sns
import matplotlib.pyplot as plt 
%matplotlib inline

In [11]:
POST_CODE_DISTANCE = 2.0 #km used to query dataset

Version: 1 

Goal: Test and eval webservice model

Data description:
- 1127 postcodes
- Google POI + EPC statistics + Property Data

Goal: sqf prices

In [12]:
# Execute this cell paste the Key, and press the key ENTER...
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


Data with PropertyData information

In [13]:
DATA = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Ben/datasetV6.csv') 

Data without PropertyData information

In [14]:
postcodesData = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Ben/googlePlaces.csv') 

In [15]:
len(DATA)

1127

In [16]:
DATA.head()

,Unnamed: 0,postcode,synagogue_rating_mean_1500,synagogue_rating_std_1500,synagogue_count_1500,synagogue_rating_mean_1000,synagogue_rating_std_1000,synagogue_count_1000,synagogue_rating_mean_500,synagogue_rating_std_500,...,dyn_rents_70pc_rangeMAX,dyn_rents_80pc_rangeMIN,dyn_rents_80pc_rangeMAX,dyn_rents_90pc_rangeMIN,dyn_rents_90pc_rangeMAX,dyn_rents_100pc_rangeMIN,dyn_rents_100pc_rangeMAX,dyn_ptal,dyn_flood_risk,dyn_mean_area
0,0,SE1 7PZ,0.0,0.0,0,0.0,0.0,0,0.0,0.0,...,750,374,800,364,850,346,1000,6b,Very Low,55.285714
1,1,SE1 7RB,0.0,0.0,0,0.0,0.0,0,0.0,0.0,...,762,381,826,375,850,346,1000,6b,Very Low,34.962963
2,2,SE1 7RD,0.0,0.0,1,0.0,0.0,0,0.0,0.0,...,762,392,826,375,850,346,1000,6b,Very Low,29.000000
3,3,SE1 7RJ,0.0,0.0,1,0.0,0.0,0,0.0,0.0,...,750,381,800,375,850,346,1000,6b,Very Low,15.689655
4,4,SE1 7SP,0.0,0.0,0,0.0,0.0,0,0.0,0.0,...,1350,415,1450,374,1950,283,5000,6a,Very Low,1043.000000


In [17]:
len(postcodesData)

10334

In [18]:
postcodesData.head()

,Unnamed: 0,postcode,synagogue_rating_mean_1500,synagogue_rating_std_1500,synagogue_count_1500,synagogue_rating_mean_1000,synagogue_rating_std_1000,synagogue_count_1000,synagogue_rating_mean_500,synagogue_rating_std_500,...,construction_2007_2011,construction_2012_onwards,postcode_Latitude,postcode_Longitude,MSOA_Code,LSOA_Code,Distance_to_station,Average_Income,Population,Households
0,0,SE1 7PQ,0.0,0.0,1,0.0,0.0,1,0.0,0.0,...,0.002759,0.0,51.503655,-0.115774,E02006801,E01032582,0.182891,53600,NaN,NaN
1,1,SE1 7PT,0.0,0.0,0,0.0,0.0,0,0.0,0.0,...,0.000000,0.0,51.496823,-0.110898,E02006801,E01003014,0.263302,53600,23.0,10.0
2,2,SE1 7PU,0.0,0.0,1,0.0,0.0,0,0.0,0.0,...,0.002759,0.0,51.499084,-0.113369,E02006801,E01003013,0.110848,53600,NaN,NaN
3,3,SE1 7PX,0.0,0.0,0,0.0,0.0,0,0.0,0.0,...,0.002759,0.0,51.496525,-0.111372,E02006801,E01003014,0.290737,53600,NaN,NaN
4,4,SE1 7PY,0.0,0.0,1,0.0,0.0,1,0.0,0.0,...,0.002759,0.0,51.502631,-0.116969,E02006801,E01003013,0.262916,53600,NaN,NaN


In [20]:
def train_test_itemData():
  def near(latP,lagP,dist):
    loc1 = (latP,lagP)
    loc2 = (lat,lng)
    
    return hs.haversine(loc1,loc2) < dist
  
  rand_index = random.randint(0, len(postcodesData))
  lat = postcodesData.iloc[rand_index]['postcode_Latitude']
  lng = postcodesData.iloc[rand_index]['postcode_Longitude']
  postcode = postcodesData.iloc[rand_index]['postcode']

  nearPostCode = set(postcodesData[postcodesData[['postcode_Latitude','postcode_Longitude']].apply(lambda x: near(x['postcode_Latitude'],x['postcode_Longitude'],POST_CODE_DISTANCE),axis=1)]['postcode'])
  Data = DATA.copy()
  Data = Data[Data['postcode'].apply(lambda x: x in nearPostCode)]
  # Remove from dataset postcode to be predicted
  item = Data[Data['postcode']==postcode]
  item_idex = item.index
  Data.drop(item_idex, inplace = True)
  # Crime
  Data['crime_total']=Data['PublicOrder']+Data['Burglary']+Data['Robbery']+Data['PossessionOfWeapons']+Data['BicycleTheft']+Data['AntiSocialBehaviour']+Data['Violence']+Data['Theft']+Data['Shoplifting']+Data['OtherCrime']+Data['Drugs']+Data['OtherTheft']+Data['VehicleCrime']+Data['CriminalDamage']
  Data['PublicOrder']=Data['PublicOrder']/Data['crime_total']
  Data['Burglary'] = Data['Burglary']/Data['crime_total']
  Data['Robbery'] = Data['Robbery']/Data['crime_total']
  Data['PossessionOfWeapons'] = Data['PossessionOfWeapons']/Data['crime_total']
  Data['BicycleTheft'] = Data['BicycleTheft']/Data['crime_total']
  Data['AntiSocialBehaviour'] = Data['AntiSocialBehaviour']/Data['crime_total']
  Data['Violence'] = Data['Violence']/Data['crime_total']
  Data['Theft'] = Data['Theft']/Data['crime_total']
  Data['Shoplifting'] = Data['Shoplifting']/Data['crime_total']
  Data['OtherCrime'] = Data['OtherCrime']/Data['crime_total']
  Data['Drugs'] = Data['Drugs']/Data['crime_total']
  Data['OtherTheft'] = Data['OtherTheft']/Data['crime_total']
  Data['VehicleCrime'] = Data['VehicleCrime']/Data['crime_total']
  Data['CriminalDamage'] = Data['CriminalDamage']/Data['crime_total']

  Data['crime_Type_A'] = Data['Robbery']+Data['Drugs']+Data['PossessionOfWeapons']+Data['Burglary']+Data['OtherCrime']
  Data['crime_Type_B'] = Data['OtherTheft'] + Data['Shoplifting'] + Data['Theft'] + Data['BicycleTheft']
  Data['crime_Type_C'] = Data['CriminalDamage']+ Data['Violence'] + Data['AntiSocialBehaviour']
  Data['crime_Type_D'] = Data['VehicleCrime']
  dropList=['Burglary',
  'Robbery',
  'PossessionOfWeapons',
  'BicycleTheft',
  'AntiSocialBehaviour',
  'Violence',
  'Theft',
  'Shoplifting',
  'OtherCrime',
  'Drugs',
  'OtherTheft',
  'VehicleCrime',
  'CriminalDamage',]
  Data.drop(dropList, axis = 1, inplace = True)
  #Age
  Data['TotalModPop'] = Data['age20_24']+Data['age25_29']+Data['age30_34']+Data['age35_39']+Data['age40_44']+Data['age45_49']+Data['age45_49']+Data['age50_54']+Data['age60_64']
  Data['age20_29'] = Data['age20_24']+Data['age25_29']
  Data['age30_64'] = Data['age30_34']+Data['age35_39']+Data['age40_44']+Data['age45_49']+Data['age50_54']+Data['age55_59']+Data['age60_64']
  Data['age65_'] = Data['age65_69']+Data['age70_74']+Data['age75_79']+Data['age80_84']+Data['age85_89']
  dropList=['age20_24',
  'age25_29',
  'age30_34',
  'age35_39',
  'age40_44',
  'age45_49',
  'age50_54',
  'age55_59',
  'age60_64',
  'age65_69',
  'age70_74',
  'age75_79',
  'age80_84',
  'age85_89']
  Data.drop(dropList, axis = 1, inplace = True)
  Data['commute_method_public'] = Data['commute_method_train'] + Data['commute_method_bus'] + Data['commute_method_underground_light_rail']
  Data['commute_method_private'] = Data['commute_method_motorcycle'] + Data['commute_method_taxi'] + Data['commute_method_car_driver'] + Data['commute_method_car_passenger']
  Data['commute_method_open'] = Data['commute_method_foot']+Data['commute_method_bicycle']
  dropList=['commute_method_foot',
  'commute_method_bicycle',
  'commute_method_other',
  'commute_method_motorcycle',
  'commute_method_taxi',
  'commute_method_train',
  'commute_method_bus',
  'commute_method_underground_light_rail',
  'commute_method_car_driver',
  'commute_method_at_home',
  'commute_method_car_passenger',
  ]
  Data.drop(dropList, axis = 1, inplace = True)
  dropList=[ 'dyn_sold_points_analysed',
  'dyn_sold_70pc_rangeMIN',
  'dyn_sold_70pc_rangeMAX',
  'dyn_sold_80pc_rangeMIN',
  'dyn_sold_80pc_rangeMAX',
  'dyn_sold_90pc_rangeMIN',
  'dyn_sold_90pc_rangeMAX',
  'dyn_sold_sqf_points_analysed',
  'dyn_sold_sqf_70pc_rangeMIN',
  'dyn_sold_sqf_70pc_rangeMAX',
  'dyn_sold_sqf_80pc_rangeMIN',
  'dyn_sold_sqf_80pc_rangeMAX',
  'dyn_sold_sqf_90pc_rangeMIN',
  'dyn_sold_sqf_90pc_rangeMAX',
  'dyn_rents_points_analysed',
  'dyn_rents_70pc_rangeMIN',
  'dyn_rents_70pc_rangeMAX',
  'dyn_rents_80pc_rangeMIN',
  'dyn_rents_80pc_rangeMAX',
  'dyn_rents_90pc_rangeMIN',
  'dyn_rents_90pc_rangeMAX',]
  Data.drop(dropList, axis = 1, inplace = True)
  Data['HperP']=Data['Households']/Data['Population']
  #'construction_before_1900',
  #'construction_1900_1929',
  #'construction_1930_1949',
  Data['construction_1950_1975']=(Data['construction_1950_1966']+Data['construction_1967_1975']+ Data['construction_1976_1982'])
  #'construction_1976_1982',
  Data['construction_1983_1995']=(Data['construction_1983_1990']+Data['construction_1991_1995'])
  Data['construction_1996_2006']=(Data['construction_1996_2002']+Data['construction_2003_2006'])
  #'construction_2007_2011'
  #'construction_2012_onwards'
  dropList=[
    #'construction_before_1900',
    #'construction_1900_1929',
    #'construction_1930_1949',
    'construction_1950_1966',
    'construction_1967_1975',
    #'construction_1976_1982',
    'construction_1983_1990',
    'construction_1991_1995',
    'construction_1996_2002',
    'construction_2003_2006',
    #'construction_2007_2011',
    #'construction_2012_onwards'
  ]
  Data.drop(dropList, axis = 1, inplace = True)
  dropList=['Unnamed: 0','postcode','count',
          'postcode_Latitude','postcode_Longitude','MSOA_Code','LSOA_Code','area'
            ]
  Data.drop(dropList, axis = 1, inplace = True)
  dropList=['dyn_sold_average',
  'dyn_sold_100pc_rangeMIN',
  'dyn_sold_100pc_rangeMAX',
  'dyn_sold_sqf_100pc_rangeMIN',
  'dyn_sold_sqf_100pc_rangeMAX',
  'dyn_rents_average',
  'dyn_rents_100pc_rangeMIN',
  'dyn_rents_100pc_rangeMAX',
  'dyn_ptal', 
    'dyn_flood_risk',
    'crime_rating'
    ]
  Data.drop(dropList, axis = 1, inplace = True)
  #sq_PricePaid statistics
  mean_sold_sqf_average = Data['dyn_sold_sqf_average'].mean()
  #Define sqf_price as the model goal
  Data['price'] = Data['dyn_sold_sqf_average']
  Data.drop(['dyn_sold_sqf_average'], axis = 1, inplace = True)
  # Split data
  X, y = Data.iloc[:, :-1], Data.iloc[:, -1]
  return X,y,item,mean_sold_sqf_average

Tune Xgboost parameters selecting random postcodes using data dist < 2Km

In [21]:
def objective(trial, kf):
    # load data < 2km
    X,y,item,mean_sold_sqf_average = train_test_itemData()

    # Value Grid
    n_estimators = trial.suggest_int('n_estimators', 206, 512)
    learning_rate =trial.suggest_uniform('learning_rate', 0, 0.1) 
    #max_depth = trial.suggest_int('max_depth', 3, 10)
    min_split_loss = trial.suggest_uniform('min_split_loss', 0, 2.5)
    min_child_weight = trial.suggest_int('min_child_weight', 1, 10)
    reg_lambda = trial.suggest_uniform('reg_lambda', 0, 10)
    reg_alpha = trial.suggest_uniform('reg_alpha', 0, 10)
    colsample_bytree = trial.suggest_uniform('colsample_bytree', 0.5, 1)
    subsample = trial.suggest_uniform('subsample', 0.5, 1)
    eta = trial.suggest_uniform('eta', 0, 1)
    
    model = xg.XGBRegressor(n_estimators=n_estimators,
                          #max_depth=max_depth,
                          learning_rate=learning_rate,
                          min_split_loss=min_split_loss,
                          min_child_weight=min_child_weight,
                          reg_lambda=reg_lambda,
                          reg_alpha=reg_alpha,
                          colsample_bytree=colsample_bytree,
                          subsample=subsample,
                          eta=eta,
                          objective='reg:squarederror', 
                          booster='gbtree',
                          seed=42,
                          n_jobs=3)
    
    
    test_score = []
    
    step = 0
    
    for train_index, test_index in kf.split(X, y):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        
        model.fit(X_train, y_train)
        
        y_pred = model.predict(X_test)
        
        test_score.append(MAE(y_test, y_pred)/mean_sold_sqf_average*100)
        
        trial.report(np.mean(test_score), step)
        
        if trial.should_prune():
            raise optuna.TrialPruned()
            
        step+=1

    return np.mean(test_score)



kf = KFold(n_splits=4, shuffle=True)

study = optuna.create_study(direction='minimize',
                            pruner=optuna.pruners.SuccessiveHalvingPruner())

study.optimize(lambda trial: objective(trial, kf), timeout=30*60, n_jobs=-1)

study.best_params 

[I 2022-03-31 02:08:06,780] A new study created in memory with name: no-name-d0706532-88bd-475c-ab48-aed9c18eb5cc
/usr/local/lib/python3.7/dist-packages/optuna/study/study.py:397: FutureWarning: `n_jobs` argument has been deprecated in v2.7.0. This feature will be removed in v4.0.0. See https://github.com/optuna/optuna/releases/tag/v2.7.0.
  FutureWarning,
[I 2022-03-31 02:08:13,428] Trial 1 finished with value: 2.073156914394143 and parameters: {'n_estimators': 449, 'learning_rate': 0.09379226047164624, 'min_split_loss': 1.7417948284674782, 'min_child_weight': 8, 'reg_lambda': 5.855210169552665, 'reg_alpha': 1.7838415187779866, 'colsample_bytree': 0.9332668004827461, 'subsample': 0.9652040693475492, 'eta': 0.754190942375238}. Best is trial 1 with value: 2.073156914394143.
[I 2022-03-31 02:08:15,579] Trial 2 pruned. 
[I 2022-03-31 02:08:19,162] Trial 3 pruned. 
[I 2022-03-31 02:08:22,693] Trial 4 pruned. 
[I 2022-03-31 02:08:22,730] Trial 0 finished with value: 2.1940729288779 and para

{'colsample_bytree': 0.6786662912772868,
 'eta': 0.33460910073054384,
 'learning_rate': 0.04781632156015487,
 'min_child_weight': 10,
 'min_split_loss': 1.3630245222110096,
 'n_estimators': 451,
 'reg_alpha': 7.337147736476194,
 'reg_lambda': 1.2863763523217604,
 'subsample': 0.56564819242061}

In [22]:
model = XGBRegressor(objective='reg:squarederror', 
                     n_estimators= 451,
                     learning_rate= 0.04781632156015487,
                     min_split_loss= 1.3630245222110096,
                     min_child_weight= 10,
                     reg_lambda= 1.2863763523217604,
                     reg_alpha= 7.337147736476194,
                     colsample_bytree= 0.6786662912772868,
                     subsample= 0.56564819242061,
                     eta= 0.33460910073054384)

In [23]:
Train_MAE = []
Test_MAE = []
Train_MAPE = []
Test_MAPE = []
Train_R2 = []
Test_R2 = []
num_tests = 100 # number of codepost selected to evaluate

In [24]:
for it in range(num_tests):
  # Data
  X,y,item,mean_sold_sqf_average = train_test_itemData()
  # Splitting
  train_X, test_X, train_y, test_y = train_test_split(X, y, 
                        test_size = 0.2)

  xgb_r = model.fit(train_X, train_y)

  pred_train = model.predict(train_X)
  pred_test = model.predict(test_X)

  # MAE Computation
  mae =MAE(train_y, pred_train)
  Train_MAE.append((mae,mae/mean_sold_sqf_average*100))
  # MAE Computation
  mae =MAE(test_y, pred_test)
  Test_MAE.append((mae,mae/mean_sold_sqf_average*100))

  # MAE Computation
  mape =MAPE(train_y, pred_train)
  Train_MAPE.append((mape,mape/mean_sold_sqf_average*100))
  # MAE Computation
  mape =MAPE(test_y, pred_test)
  Test_MAPE.append((mape,mape/mean_sold_sqf_average*100))

  # MAE Computation
  r2 =R2(train_y, pred_train)
  Train_R2.append(r2)

  # MAE Computation
  r2 =R2(test_y, pred_test)
  Test_R2.append(r2)

In [25]:
def mean_list(l,n):
  aux1=0
  aux2=0
  for v1,v2 in l:
    aux1+=v1
    aux2+=v2
  return (aux1/n,aux2/n)

In [26]:
#MAE Computation
print("Train MAE : % f --- %f" %mean_list(Train_MAE,num_tests),"%")

# MAE Computation
print("Test MAE : % f --- %f" %mean_list(Test_MAE,num_tests),"%")

# MAE Computation
print("Train MAPE : % f --- %f" %mean_list(Train_MAPE,num_tests),"%")

# MAE Computation
print("Test MAPE : % f --- %f" %mean_list(Test_MAPE,num_tests),"%")

# MAE Computation
print("Train R2 : % f" %(sum(Train_R2)/num_tests))

# MAE Computation
print("Test R2 : % f" %(sum(Test_R2)/num_tests))

Train MAE :  3.398796 --- 0.466732 %
Test MAE :  16.087907 --- 2.229156 %
Train MAPE :  2.596173 --- 0.357761 %
Test MAPE :  11.791475 --- 1.642919 %
Train R2 :  0.995161
Test R2 :  0.884313


Train MAE :  3.060648 --- 0.417311 %
Test MAE :  16.034076 --- 2.201200 %
Train MAPE :  2.317864 --- 0.316704 %
Test MAPE :  11.487136 --- 1.588041 %
Train R2 :  0.996488
Test R2 :  0.872903